In [ ]:
raise("Deprecated")
import pandas as pd 
import numpy as np

In [ ]:
inceid=pd.read_csv('../../../results/Process_missingness/baseline_imputed.csv')['eid'].to_numpy()

In [ ]:
np.concatenate(pd.read_csv('../../results/phecode130.csv').to_numpy())

In [ ]:
import os
icdmap=dict(zip(pd.read_csv('../../../data/phecode_icd10.csv')['ALT_CODE'].to_numpy(),pd.read_csv('../../../data/phecode_icd10.csv')['PheCode'].to_numpy()))
phe=np.concatenate(pd.read_csv('../../../data/phecode.csv').to_numpy())
valid={}
for k,v in icdmap.items():
    if v in phe:
        valid[k]=v
valid

In [ ]:
validX={}
for k,v in icdmap.items():
    if 'X' == k[-1]:
        if v in list(phe):
            validX[k[:-1]]=v 
validX

In [ ]:
def icdtophe(icd):
    if icd in valid.keys():
        return True,valid[icd]
    elif icd[:-3] in validX.keys():
        return True,validX[icd[:-3]]
    else:
        return False,None
        

In [ ]:
cd3=pd.read_csv('../../../data/cd3.tsv',delimiter='\t')
icdmapnonalt=dict(zip(pd.read_csv('../../../data/phecode_icd10.csv')['ICD10'].to_numpy(),pd.read_csv('../../../data/phecode_icd10.csv')['PheCode'].to_numpy()))
phemeaning=dict(zip(pd.read_csv('../../../data/phecode_definitions1.2.csv')['phecode'].to_numpy(),pd.read_csv('../../../data/phecode_definitions1.2.csv')['phenotype'].to_numpy()))


In [ ]:
icdmapnonalt

In [ ]:
def reslove_intv(intv):
    inc=[]
    start,end=intv.split('-')
    for k,v in icdmapnonalt.items():
        if start<=k<=end:
            inc.append(icdmapnonalt[k])
    if len(inc)==0:
        if intv[:3] =='C07':
            inc=[icdmapnonalt[i[2][:3]]]
    if len(inc)==0:
        print(i)
    #print(intv,inc)
    return inc
def reslove_code(cd):
    phe=''
    if cd in icdmapnonalt.keys():
        phe=icdmapnonalt[cd]
    elif cd[:3] in icdmap.keys():
        phe=icdmap[i[2][:3]]
    elif cd[:3] in icdmapnonalt.keys():
        phe=icdmapnonalt[i[2][:3]]
    else:
        phe=np.nan
        if False:#'.' not in cd:
            cd=cd+'.0'
            phe=reslove_code(cd)[0]
        else:
            print(cd)
    return [phe]
cdcancer={}
mDF=[]
for i in cd3.to_numpy():
    if '-' not in i[2]:
        res=reslove_code(i[2])
    elif ',' not in i[2]:
        res=reslove_intv(i[2])
    else:
        res=[]
        for itm in i[2].split(', '):
            if '-' not in itm:
                res+=reslove_code(itm)
            else:
                res+=reslove_intv(itm)
    res=np.unique(res)
    incres=[]
    for t in res:
        if t in phe:
            incres.append(t)
    for t in incres:
        mDF.append([i[0],i[1],t,phemeaning[t]])
    ####
    cdcancer[i[0]]=incres[0]

In [ ]:
mDF

In [ ]:
pd.DataFrame(mDF).to_csv('../../../data/cancermap.csv')

In [ ]:
include='''20001
20002
40001
41270'''.split('\n')
include

In [ ]:
cod6=pd.read_csv('../../../data/Self_report_FO_mappings_Jan2022.tsv',delimiter='\t')

In [ ]:
cod6icd={}
for i in cod6.to_numpy():
    cod6icd[i[0]]=i[1]

In [ ]:
cod6icd

In [ ]:
includewtime='''40001,40007
41270,41280
20001,20007
20002,20009'''.split('\n')
includewtime

In [ ]:
def handle_code(datai,data):
    status=False
    if data=='20002':
        if int(float(datai)) in cod6icd.keys():
            status,code=icdtophe(cod6icd[int(float(datai))])
    elif data=='20001':
        if int(float(datai)) in cdcancer.keys():
            status=True
            code=cdcancer[int(float(datai))]
    else:
        status,code=icdtophe(datai)
    if status:
        return status,code
    else:
        return status,None

handle_code(1002.0,'20001')

In [ ]:
overall={}
for i in includewtime:
    data,time=i.split(',')
    DF=[]
    for i in sorted(os.listdir('../../../results/cache/ukbcsv/')):
        if data in i:
            DF.append(pd.read_csv('../../../results/cache/ukbcsv/'+i,header=None))
    dataDF=pd.concat(DF,axis=1)
    DF=[]
    for i in sorted(os.listdir('../../../results/cache/ukbcsv/')):
        if time in i:
            DF.append(pd.read_csv('../../../results/cache/ukbcsv/'+i,header=None))
    timeDF=pd.concat(DF,axis=1)
    tnp=timeDF.to_numpy()
    dnp=dataDF.to_numpy()
    perfiledisease=[]
    for j in range(tnp.shape[0]):
        disedict={}
        for i in range(tnp.shape[1]):
            timei=str(tnp[j,i])
            datai=str(dnp[j,i])
            if timei!='nan' and datai!='nan':
                status,code=handle_code(datai,data)
                if not status:#not in our range
                    continue
                try:
                    if float(timei)<0:#error code
                        continue
                except ValueError:#41270 text
                    pass
                datai=code
                if datai in disedict.keys():
                    if timei<disedict[datai]:
                        disedict[datai]=timei
                else:
                    disedict[datai]=timei
        perfiledisease.append(disedict)
    overall[data]=perfiledisease

In [ ]:
import pickle
pricare=pickle.load( open('../../../results/cache/pricare', 'rb'))

In [ ]:
fulleid=np.concatenate(pd.read_csv('../../../results/cache/ukbcsv/eid.csv',header=None).to_numpy())
fulleid=list(fulleid)

In [ ]:
priprocessed=[]
for k in fulleid:
    if k not in pricare.keys():
        priprocessed.append({})
    else:
        v=pricare[k]
        disedict={}
        for c,date in v.items():
            status,code=handle_code(c,'42040')
            if status:
                date=date.split('/')
                date=f'{date[2]}-{date[1]}-{date[0]}'
                datai=code
                if datai in disedict.keys():
                    if date<disedict[datai]:
                        disedict[datai]=date 
                else:
                    disedict[datai]=date 
        priprocessed.append(disedict)


In [ ]:
priprocessed

In [ ]:
import numpy as np
focc=np.load('../../../results/cache/firstocc.npy',allow_pickle=1).item()

In [ ]:
list(focc.keys())[0]

In [ ]:
foccprocessed=[]
for k in fulleid:
    if k not in pricare.keys():
        foccprocessed.append({})
    elif str(k) not in focc.keys():
        foccprocessed.append({})
    else:
        v=focc[str(k)]
        disedict={}
        for c,date in v.items():
            status,code=handle_code(c,'130000')
            if status:
                datai=code
                if datai in disedict.keys():
                    if date<disedict[datai]:
                        disedict[datai]=date 
                else:
                    disedict[datai]=date 
        foccprocessed.append(disedict)

In [ ]:
overall['42040']=priprocessed

In [ ]:
overall['130000']=foccprocessed

In [ ]:
for k,v in overall.items():
    newv=[]
    for s in v:
        noerror={}
        for tempphe,time in s.items():
            if time in ['1901-01-01','1902-02-02','1903-03-03','2037-07-07']:
                continue
            else:
                noerror[tempphe]=time
        newv.append(noerror)
    overall[k]=newv

In [ ]:
overall.keys()

In [ ]:
for k in ['40001', '41270', '20001', '20002', '42040', '130000']:
    totaltime=[]
    for i in overall[k]:
        totaltime+=list(i.values())
    if k in ['40001', '20001', '20002']:
        totaltime=np.array(totaltime,dtype=float)
    print(min(totaltime),max(totaltime))

In [ ]:
fulleid_dict = {value: index for index, value in enumerate(fulleid)}
incidx = [fulleid_dict[i] for i in inceid]
incidx

In [ ]:
f34=np.concatenate(pd.read_csv('../../../results/cache/ukbcsv/34-0.0.csv',header=None).to_numpy())

In [ ]:
f52=np.concatenate(pd.read_csv('../../../results/cache/ukbcsv/52-0.0.csv',header=None).to_numpy())

In [ ]:
f53=np.concatenate(pd.read_csv('../../../results/cache/ukbcsv/53-0.0.csv',header=None).to_numpy())

In [ ]:
ageofr=[]
agetillnow=[]
for i,d in enumerate(f53):
    ageofr.append(int(d.split('-')[0])-f34[i]+int(d.split('-')[1])/12-f52[i]/12+int(d.split('-')[2])/365)
    agetillnow.append(2024-f34[i]-f52[i]/12)
agetillnow

In [ ]:
ageofr

In [ ]:
ymd=[]
def calctime(date,year,mo):
    y,m,d=np.array(date.split('-'),dtype=int)
    return y-year+(m-mo)/12+d/365

In [ ]:
for i in range(len(overall['41270'])):
    for k,v in overall['41270'][i].items():
        overall['41270'][i][k]=calctime(v,f34[i],f52[i])

In [ ]:
for i in range(len(overall['42040'])):
    for k,v in overall['42040'][i].items():
        overall['42040'][i][k]=calctime(v,f34[i],f52[i])

In [ ]:
for i in range(len(overall['130000'])):
    for k,v in overall['130000'][i].items():
        overall['130000'][i][k]=calctime(v,f34[i],f52[i])

In [ ]:
len(overall['20002'])

In [ ]:
overall

In [ ]:
phe=list(phe)

In [ ]:
cd={}
for k,v in overall.items():
    pd={}
    print(k,len(v))
    for i in v:
        for s,t in i.items():
            if s in pd.keys():
                pd[s]+=1
            else:
                pd[s]=1
    cd[k]=pd
import csv

data = cd
output_file = '../../../results/sumdata.csv'
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Key'] + sorted(list(data.keys())))
    inner_keys = np.concatenate(pd.read_csv('../../../data/phecode.csv').to_numpy())
    for inner_key in inner_keys:
        row = [str(inner_key)]
        for key in data.keys():
            row.append(data[key].get(inner_key, ''))
        writer.writerow(row)


In [ ]:

for k in overall.keys():
    matrix=[]
    total=0
    for i in range(502244):
        temp=np.zeros((len(phe),))
        v=overall[k]
        add=0
        for code,time in v[i].items():
            add=1
            dindex=phe.index(code)
            time=float(time)
            if time-ageofr[i]>0:
                temp[dindex]=1
            else:
                temp[dindex]=np.nan
        total+=add
        matrix.append(temp)
    print(k,total,np.nansum(np.array(matrix)))

In [ ]:
for k in overall.keys():
    matrix=[]
    total=0
    for i in range(502244):
        temp=np.zeros((len(phe),))
        v=overall[k]
        add=0
        for code,time in v[i].items():
            add=1
            dindex=phe.index(code)
            time=float(time)
            if time-ageofr[i]>0:
                temp[dindex]=1
            else:
                temp[dindex]=1
        total+=add
        matrix.append(temp)
    print(k,total,np.nansum(np.array(matrix)))

In [ ]:
matrix=[]
for i in range(502244):
    print(i)
    temp=np.zeros((len(phe),))
    for k,v in overall.items():
        for code,time in v[i].items():
            dindex=phe.index(code)
            time=float(time)
            if time-ageofr[i]>0:
                temp[dindex]=1
            else:
                temp[dindex]=np.nan
    matrix.append(temp)

In [ ]:
np.nansum(matrix)

In [ ]:
matrix=[]
for i in range(502244):
    print(i)
    temp=np.zeros((len(phe),))
    for k,v in overall.items():
        for code,time in v[i].items():
            dindex=phe.index(code)
            time=float(time)
            if time-ageofr[i]>0:
                temp[dindex]=1
            else:
                temp[dindex]=1
    matrix.append(temp)

In [ ]:
np.nansum(matrix)

In [ ]:
matrix=[]
for i in range(502244):
    print(i)
    temp=np.ones((len(phe),))*agetillnow[i]
    for k,v in overall.items():
        for code,time in v[i].items():
            dindex=phe.index(code)
            time=float(time)
            if time-ageofr[i]>0:
                if temp[dindex]>time:
                    temp[dindex]=time
            else:
                temp[dindex]=np.nan
    matrix.append(temp)

In [ ]:
matrix0_1direct=[]
for i in range(502244):
    print(i)
    temp=np.ones((len(phe),))*agetillnow[i]
    for k,v in overall.items():
        for code,time in v[i].items():
            dindex=phe.index(code)
            time=float(time)
            if time-ageofr[i]>0:
                if temp[dindex]>time:
                    temp[dindex]=time
            else:
                temp[dindex]=np.nan
    matrix0_1direct.append(temp)

In [ ]:
matrix
matrix0_1=[]
for i in matrix:
    stop=np.nanmax(i)
    nan=np.where(np.isnan(i))[0]
    ones=np.where(i!=stop)[0]
    temp=np.zeros((len(phe),))
    temp[ones]=1
    temp[nan]=np.nan
    matrix0_1.append(temp)

In [ ]:
matrix0_1=np.array(matrix0_1)

In [ ]:
fulleid_dict = {value: index for index, value in enumerate(fulleid)}
incidx = [fulleid_dict[i] for i in inceid]
print(incidx)
incidx=np.array(incidx)

In [ ]:
np.save('../../../results/cache/0-1',matrix0_1[incidx,:])

In [ ]:
np.save('../../../results/cache/cox',np.array(matrix)[incidx,:])